# VI: Second Practical Work

**Authors:** Gerard Comas & Marc Franquesa.

In [ ]:
import pandas as pd
import numpy as np
import altair as alt
import geopandas as gpd
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

collisions = pd.read_csv("./processed-data/collisions.csv")
# map_data = gpd.read_file("./processed-data/map.geojson")
weather = pd.read_csv("./processed-data/weather.csv")

primary = "green"
secondary = ""

colors = {
    primary: "#66c2a4",
    secondary: "",
}

collisions.head()

La nostre idea per a respondre a totes les preguntes d'aquest segon projecte es crear un Dashboard interactiu que tindrà la distribució següent:
- En la primera fila tres barplots en el que es representen el total de colisions amb un filtre concret. Aquests tres barplots seràn independents. Els tres barplots compartiran l'escala i l'escala de colors per tal de poder apreciar millor les magnituds. 
    - El primer de tots serà un barplot amb el total de colisions per cada mes, amb la funcionalitat de poder seleccionat una de les barres i els gràfics de les altre files dependeràn d'aquest filtrat. Si no es selecciona cap es mostraran tots els mesos. 
    - El segon serà un barplot amb el total de colisions per cada tipus de vehicle, amb la mateixa funcionalitat que abans.
    - El tercer i últim serà un barplot amb el total de colisions per a cada condició climàtica, amb la mateia funcionalitat que abans. 
- En la segona fila trobarem un element:
    - Un mapa de New York, on primerament es mostrin les fronteres dels barris. Aquest mapa dera un chloropleth i l'intensitat del color indicarà el total de colisions pel filtrat seleccionat. 
        - Hem de decidir si l'escala serà independent o no del total
        - També, podriem afegir més granularitat temporal amb un desplegable per a canviar.
        - El mapa incorporarà un tooltip que ens identifiqui la localitació i també el nombre de colisions
        - A més, serà interactiu i pemetrà filtrar per als gràfics de les lines inferiors.
- En la tercera fila trobarem dos elements:
    - Un heatmap on les files indiquen les setmanes i a les columnes els dies de la setmana. Està dissenyat per a que recordi a un calendari. Quan no s'hagi seleccionat el mes es mostrarà unes opcions per a escollir el mes a representar. El color indicarà la quantitat de colisions aquell dia.
    - En el segon element trobarem un line chart, aquest mostrarà totes les línes per a cada area dels mesos seleccionats, si s'ha seleccionat un mes, només es mostrarnan la d'aquell mes i si s'ha seleccionat una area es resaltarà aquella linea per sobre les altres. A més, hi haurà un slider per a marcar quina hora volem analitar, indicant-nos amb punts sobre les lines els valors per a cada area i per tant poder obtenir els valors més facilment. 

Pot ser confús per l'usuari doncs no queda del tot clar quines gràfiques filtren i quines no igual que la dependencia entre gràfics. 

In [ ]:
collisions["MONTH"] = pd.to_datetime(collisions["CRASH DATETIME"]).dt.strftime("%B")


monthly_collisions = collisions.groupby("MONTH").size().reset_index(name="counts")
months_order = ["June", "July", "August", "September"]
month_selection = alt.selection_point(fields=["MONTH"], empty=True)
vehicle_order = ["Taxi", "Ambulance", "Fire truck"]
vehicle_selection = alt.selection_point(fields=["VEHICLE"], empty=True)

months = (
    alt.Chart(monthly_collisions)
    .mark_bar(color=colors[primary])
    .encode(
        x=alt.X("MONTH:O", sort=months_order, axis=alt.Axis(title="Month", labelAngle=0)),
        y=alt.Y("counts:Q", axis=alt.Axis(title="Collisions")),
        opacity=alt.condition(month_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("counts:Q", title="Collisions")],
    ).add_params(
        month_selection
    ).properties(width=250, height=175)
)

vehicles = (
    alt.Chart(collisions)
    .mark_bar(color=colors[primary])
    .transform_filter(month_selection)
    .encode(
        x=alt.X("VEHICLE:N", sort=vehicle_order, axis=None),
        y=alt.Y("count():Q", axis=alt.Axis(title="Collisions")),
        opacity=alt.condition(vehicle_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("VEHICLE:N", title="Vehicle"), alt.Tooltip("count():Q", title="Collisions")],
    ).add_params(
        vehicle_selection
    ).properties(width=100, height=175)
)

emojis = (
    alt.Chart(collisions)
    .mark_text(size=18, align="center", dy=-8)
    .transform_filter(month_selection)
    .encode(
        x=alt.X("VEHICLE:N"),
        y=alt.Y("count():Q"),
        text=alt.Text("VEHICLE EMOJI:N"),
        opacity=alt.condition(vehicle_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("VEHICLE:N", title="Vehicle"), alt.Tooltip("count():Q", title="Collisions")],
    ).add_params(
        vehicle_selection
    )
)

months | (vehicles + emojis)